In [ ]:
import sys, os, pathlib
project_root = pathlib.Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.graph import nxgraph
import src.agent as agent_module
from utils.plotter import draw_graph, draw_graph_with_transfers, draw_request_phase, draw_response_phase, draw_split_round
from utils.simulator import simulate_round, get_network_stats

# Create and setup the graph (ER or BA)
G = nxgraph()
# G.ER_graph(nodes=20, prob=0.1, weighted=True, lower_ut=10, upper_ut=100)
G.BA_graph(nodes=10, edges=2, seed=42, weighted=True, lower_ut=10, upper_ut=100)

agent_module.assign_n_seeders(G.graph, n=1, seed=42)
FILE_PIECES = 15
print("Initial graph setup:")
print(f"Graph has {G.graph.number_of_nodes()} nodes and {G.graph.number_of_edges()} edges")

draw_graph(G.graph, total_pieces=FILE_PIECES)


In [ ]:
# Init
agent_module.initialize_file_sharing(G.graph, FILE_PIECES, seed=42)
print(str(FILE_PIECES) + " pieces")
for node in G.graph.nodes():
    info = agent_module.get_agent_info(G.graph, node)
    print(f"Node {node}: {info['role']} - {info['num_pieces']}/ {FILE_PIECES} pieces - Complete: {info['is_complete']}")

print(f"\nTotal pieces in network at the moment: {sum(len(G.graph.nodes[node].get('file_pieces', set())) for node in G.graph.nodes())}")


In [ ]:
stats = get_network_stats(G.graph, FILE_PIECES)
print(f"Initial: {stats['incomplete_leechers']} leechers incomplete, {stats['completion_rate']:.1%} completion rate")

# Run one round of simulation
for round_num in range(1, 30):
    print(f"\n Round {round_num}")
    result = simulate_round(G.graph, FILE_PIECES, seed=42+round_num)
    
    print(f"Transfers: {result['total_transfers']}")
    if result['new_completions']:
        print(f"New completions: {result['new_completions']}")
    
    stats = get_network_stats(G.graph, FILE_PIECES)
    print(f"Progress: {stats['incomplete_leechers']} leechers incomplete, {stats['completion_rate']:.1%} completion rate")
    
    # Show some transfer details
    if result['transfers']:
        print("transfers:")
        for transfer in result['transfers']:  # Show first handful
            print(f"  Piece {transfer['piece']}: {transfer['from']} to >> {transfer['to']}")
            # save_images=True
    
    # Show both phases side by side
    draw_split_round(G.graph, result['requests'], result['transfers'], 
                    FILE_PIECES, round_num, save_images=False)
    
 # Show phases separately (uncomment to use)
   # draw_request_phase(G.graph, result['requests'], FILE_PIECES, round_num, save_images=False)
   # draw_response_phase(G.graph, result['transfers'], FILE_PIECES, round_num, save_images=False)
print(f"\nFinal network state:")
for node in G.graph.nodes():
    info = agent_module.get_agent_info(G.graph, node)
    print(f"Node {node}: {info['role']} - {info['num_pieces']}/20 pieces - Complete: {info['is_complete']}")

final_stats = get_network_stats(G.graph, FILE_PIECES)

print(f"\nFinal network statistics:")
print(f"Total nodes: {final_stats['total_nodes']}")
print(f"Total edges: {final_stats['total_edges']}")
print(f"Seeders: {final_stats['seeders']}")
print(f"Leechers: {final_stats['leechers']}")
print(f"Complete leechers: {final_stats['complete_leechers']}")
print(f"Incomplete leechers: {final_stats['incomplete_leechers']}")
print(f"Completion rate: {final_stats['completion_rate']:.1%}")
print(f"Total pieces in network: {final_stats['total_pieces_in_network']}")

# Draw the final graph
print(f"\nFinal graph visualization:")
draw_graph(G.graph, total_pieces=FILE_PIECES)